In [91]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import exp
from datetime import datetime

DATA_FILE = "./Data/daily_DIS.csv"
RSI_INTERVAL = 10
ALMA_WINDOW = 18
RSI_LONG_ENTRY = 30
RSI_SHORT_ENTRY = 70
TP = 0.02  # Take profit: 0.8%
SL = 0.005  # Stop loss: 0.5%

In [92]:
def load_file(file_path):
    df = pd.read_csv(file_path, parse_dates=['timestamp'], date_format="%d/%m/%Y")
    df.sort_values(by='timestamp', inplace=True)
    return df

df = load_file(DATA_FILE)

print(df.head())
print(df.tail())

       timestamp    open    high     low   close    volume
1766  2018-01-02  108.95  111.81  108.56  111.80  10989294
1765  2018-01-03  112.19  113.19  111.45  112.28   9207141
1764  2018-01-04  112.95  113.00  111.63  112.23   7400598
1763  2018-01-05  112.68  112.68  111.24  111.62   5989794
1762  2018-01-08  110.89  111.28  109.54  110.02   8038517
    timestamp    open      high     low   close   volume
4  2025-01-03  111.37  111.5350  110.18  111.16  5394318
3  2025-01-06  111.47  112.8537  110.87  111.05  6274419
2  2025-01-07  112.15  113.7400  111.29  111.39  7878807
1  2025-01-08  111.00  111.1100  108.64  109.76  7805312
0  2025-01-10  110.00  110.5400  107.62  108.65  8835518


In [93]:
def add_rsi(df, interval):
    delta = df["close"].diff()

    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0

    _gain = up.ewm(com=(interval - 1), min_periods=interval).mean()
    _loss = down.abs().ewm(com=(interval - 1), min_periods=interval).mean()

    RS = _gain / _loss
    df['rsi'] = 100 - (100 / (1 + RS))
    return df

df = add_rsi(df, RSI_INTERVAL)

In [94]:
def calculate_alma(prices: list, window: int = ALMA_WINDOW, offset: float = 0.85, sigma: float = 6) -> float:
    m = int(offset * (window - 1))
    s = window / sigma
    weights = np.array([exp(-((k - m) ** 2) / (2 * (s ** 2))) for k in range(window)])
    
    if len(prices) < window:
        return None
    weighted_sum = weights * prices[-window:]
    alma = weighted_sum.sum() / weights.sum()
    return alma

def add_alma_column(df: pd.DataFrame, window: int = ALMA_WINDOW) -> pd.DataFrame:
    alma_values = []
    for i in range(len(df)):
        prices = df['rsi'][:i+1].values
        alma = calculate_alma(prices, window)
        alma_values.append(alma)    

    df['alma'] = alma_values
    return df

df = add_alma_column(df)

In [95]:
profit_loss_log = []
trade_log = []
open_positions = []
total_trades = 0
winning_trades = 0
days_open_position = 0
format = "%Y-%m-%d"  # Define the format

with open("trade_log.txt", "w") as log_file:
    log_file.write("Trade Log\n")
    log_file.write("=" * 40 + "\n")

    for i, row in df.iterrows():
        close_price = row["close"]
        current_date = row["timestamp"] 
        rsi = row["rsi"]

        updated_positions = []
        for position in open_positions:
            if position["type"] == "LONG":
                tp_price = position["entry"] * (1 + TP)
                sl_price = position["entry"] * (1 - SL)
                if close_price >= tp_price or close_price <= sl_price:
                    # Calculate profit/loss
                    result = (
                        (tp_price - position["entry"]) / position["entry"] * 100
                        if close_price >= tp_price
                        else (sl_price - position["entry"]) / position["entry"] * 100
                    )
                    profit_loss_log.append(result)
                    trade_log.append(
                        f"Trade {total_trades + 1}: Entered LONG @ {position['entry']} on {position['entry_date']} "
                        f"and EXITED @ {close_price} on {current_date} for a profit/loss of {result:.2f}%\n"
                    )
                    delta_days = datetime.strptime(current_date, format) - datetime.strptime(position['entry_date'], format) 
                    days_open_position += delta_days.days
                    log_file.write(trade_log[-1])
                    total_trades += 1
                    if result > 0:
                        winning_trades += 1
                else:
                    updated_positions.append(position)
            elif position["type"] == "SHORT":
                tp_price = position["entry"] * (1 - TP)
                sl_price = position["entry"] * (1 + SL)
                if close_price <= tp_price or close_price >= sl_price:
                    # Calculate profit/loss
                    result = (
                        (position["entry"] - tp_price) / position["entry"] * 100
                        if close_price <= tp_price
                        else (position["entry"] - sl_price) / position["entry"] * 100
                    )
                    profit_loss_log.append(result)
                    trade_log.append(
                        f"Trade {total_trades + 1}: Entered SHORT @ {position['entry']} on {position['entry_date']} "
                        f"and EXITED @ {close_price} on {current_date} for a profit/loss of {result:.2f}%\n"
                    )
                    delta_days = datetime.strptime(current_date, format) - datetime.strptime(position['entry_date'], format) 
                    days_open_position += delta_days.days
                    log_file.write(trade_log[-1])
                    total_trades += 1
                    if result > 0:
                        winning_trades += 1
                else:
                    updated_positions.append(position)

        open_positions = updated_positions

        # Check for new trades
        if rsi < RSI_LONG_ENTRY:
            open_positions.append(
                {"type": "LONG", "entry": close_price, "entry_date": current_date}
            )
        elif rsi > RSI_SHORT_ENTRY:
            open_positions.append(
                {"type": "SHORT", "entry": close_price, "entry_date": current_date}
            )

    win_rate = (winning_trades / total_trades) * 100 if total_trades > 0 else 0

    log_file.write("\nSummary Statistics\n")
    log_file.write("=" * 40 + "\n")
    log_file.write(f"Total Trades: {total_trades}\n")
    log_file.write(f"Win Rate: {win_rate:.2f}%\n")
    log_file.write(f"Total Profit/Loss: {sum(profit_loss_log):.2f}%\n")
    # the avg p/l is also equivalent to total p/l divided by total trades
    log_file.write(f"Avg P/L per Trade: {(win_rate * TP - (100-win_rate) * SL ):.2f}%\n")
    log_file.write(f"Avg Time in Trade: {days_open_position / total_trades :.2f} Days\n")
    log_file.write(f"Underlying Asset Hold P/L: {(df["close"].iloc[-1] - df["close"].iloc[0])/(df["close"].iloc[0]) * 100 : .2f}%")


rough idea: use past longer period standard dev to identify if market is sideways, then use RSI to entry exit